In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Q1.  Who are the top 5 users in Chicago with the highest number of followers? List their login in order, comma-separated

In [3]:
users = pd.read_csv('users.csv')
users.head()

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
0,cassidoo,Cassidy Williams,GITHUB,"Chicago, IL",NaN,NaN,Making memes and dreams... and software,165,13373,102,2012-02-20T16:36:23Z
1,felangel,Felix Angelov,SHOREBIRDTECH,Chicago,felangelov@gmail.com,NaN,"software engineer by day, software engineer by...",125,8669,67,2014-09-22T02:35:58Z
2,dabeaz,David Beazley,"DABEAZ, LLC",Chicago,dave@dabeaz.com,NaN,Author of the Python Essential Reference (Addi...,34,5179,0,2010-08-01T15:22:48Z
3,sstephenson,Sam Stephenson,NaN,Chicago,sam@sls.name,NaN,NaN,24,3761,0,2008-03-08T22:17:24Z
4,mattgodbolt,Matt Godbolt,AQUATIC CAPITAL MANAGEMENT,"Chicago, IL",matt@godbolt.org,NaN,"Compiler Explorer and jsbeeb creator, ex-Googl...",84,3397,97,2011-02-23T13:46:48Z


In [4]:
users['hireable'] = users['hireable'].fillna(False).astype(bool)

<ipython-input-4-6ea7a0647cbf>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  users['hireable'] = users['hireable'].fillna(False).astype(bool)


In [5]:
top5 = users.sort_values(by='followers', ascending=False).head()
print(','.join(top5['login'].tolist()))

cassidoo,felangel,dabeaz,sstephenson,mattgodbolt


#### Q2. Who are the 5 earliest registered GitHub users in Chicago? List their login in ascending order of created_at, comma-separated.

In [6]:
users['created_at'] = pd.to_datetime(users['created_at'])

In [7]:
top_earliest = users.sort_values(by='created_at').head()
print(','.join(top_earliest['login'].tolist()))

ELLIOTTCABLE,trevorturk,lukehoersten,djspiewak,shanesveller


#### Q3. What are the 3 most popular license among these users? Ignore missing licenses. List the license_name in order, comma-separated.

In [9]:
repos = pd.read_csv('repositories.csv')
repos.head()

FileNotFoundError: [Errno 2] No such file or directory: 'repositories.csv'

In [12]:
repos['license_name'].value_counts().head(3)

,count
license_name,
mit,8018
other,3104
apache-2.0,2542


#### Q4. Which company do the majority of these developers work at?

In [13]:
users['company'].value_counts().head(1)

,count
company,
UNIVERSITY OF CHICAGO,12


#### Q5. Which programming language is most popular among these users?

In [14]:
repos['language'].value_counts().head(1)

,count
language,
JavaScript,5609


#### Q6. Which programming language is the second most popular among users who joined after 2020?

In [15]:
users_after_2020 = users[users['created_at'] > '2020-01-01']
users_after_2020.head()

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
36,dabeaz-course,Dabeaz LLC,NaN,Chicago,dabeaz@dabeaz.com,False,Courses by David Beazley,2,638,0,2020-05-13 16:47:19+00:00
68,vehbiu,Vehbi,NaN,"Chicago, Illinois",NaN,True,NaN,10,435,0,2023-02-22 15:56:12+00:00
77,jarry-xiao,Jarry Xiao,NaN,Chicago,NaN,False,blockchain x quant finance,23,385,10,2020-02-15 18:05:16+00:00
137,ionelabrinza,Nicole Brinza,NaN,"Chicago, USA",NaN,False,:),45,247,384,2021-10-28 16:05:46+00:00
152,syngenta-digital,Syngenta Digital,NaN,"Chicago, Pune, Belo Horizonte",developer@syngenta-digital.com,False,Syngenta Digital Product Engineering,39,228,0,2020-05-18 06:22:20+00:00


In [16]:
repos_2020 = repos[repos['login'].isin(users_after_2020['login'].tolist())]
repos_2020['language'].value_counts().head()

,count
language,
Ruby,83
JavaScript,76
Python,34
CSS,29
HTML,18


#### Q7. Which language has the highest average number of stars per repository?

In [17]:
avg_stars = repos.groupby('language')['stargazers_count'].mean()
top_lang = avg_stars.idxmax()
top_stars = avg_stars.max()
print(top_lang, top_stars)

Vim Script 647.4545454545455


#### Q8. Let's define leader_strength as followers / (1 + following). Who are the top 5 in terms of leader_strength? List their login in order, comma-separated.

In [18]:
users['leader_strength'] = users['followers'] / (1 + users['following'])
top5_lead = users.sort_values(by='leader_strength', ascending=False).head()
print(','.join(top5_lead['login'].tolist()))

dabeaz,sstephenson,khan4019,adashofdata,djspiewak


#### Q9. What is the correlation between the number of followers and the number of public repositories among users in Chicago?

In [19]:
correlation = users['followers'].corr(users['public_repos'])
correlation

0.07647335349256014

#### Q10. Does creating more repos help users get more followers? Using regression, estimate how many additional followers a user gets per additional public repository.

In [20]:
import csv
followers = []
public_repos = []
with open('users.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        followers_count = int(row['followers'])
        public_repos_count = int(row['public_repos'])
        followers.append(followers_count)
        public_repos.append(public_repos_count)
if len(followers) > 1 and len(public_repos) > 1:
    slope, intercept = np.polyfit(public_repos, followers, 1)

    print(f"{slope:.3f}")
else:
    print("Error")

0.615


#### Q14. Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated

In [29]:
import csv
from collections import Counter
from datetime import datetime

weekend_repo_counts = Counter()

with open('repositories.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)

    for row in reader:
        created_at = row.get('created_at', '')
        if created_at:
            created_date = datetime.fromisoformat(created_at[:-1])

            if created_date.weekday() in [5, 6]:
                user_login = row['login']
                weekend_repo_counts[user_login] += 1

top_users = weekend_repo_counts.most_common(5)

top_logins = [user[0] for user in top_users]

print(','.join(top_logins))

marwahaha,austinsonger,eddelbuettel,erichilarysmithsr,yyolk


#### Q16. Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.) What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)

In [25]:
new_users = users[users['name'].notna()].copy()
new_users['surname'] = new_users['name'].str.split().str[-1].str.strip()
surname_counts = new_users['surname'].value_counts()
max_count = surname_counts.max()
common_surnames = surname_counts[surname_counts == max_count].index.tolist()
common_surnames.sort()
print(','.join(common_surnames))

Smith


In [26]:
# users = users.fillna("").astype(str)
# repos = repos.fillna("").astype(str)

# repos['has_wiki'] = repos['has_wiki'].astype(str).replace({'True': 'true', 'False': 'false'})
# repos['has_projects'] = repos['has_projects'].astype(str).replace({'True': 'true', 'False': 'false'})

# # users.to_csv('users1.csv', index=False)
# repos.to_csv('repos1.csv', index=False)

### Q13. Some developers write long bios. Does that help them get more followers? What's the impact of the length of their bio (in Unicode words, split by whitespace) with followers? (Ignore people without bios)

###Regression slope of followers on bio word count (to 3 decimal places, e.g. 12.345 or -12.345)

In [1]:
import pandas as pd
import statsmodels.api as sm

# Load the CSV file
csv_file = 'users.csv'  # Ensure this path is correct

# Load the CSV into a DataFrame
df = pd.read_csv(csv_file)

# Check the first few rows and the data types of the DataFrame
print("DataFrame Overview:")
print(df.head())
print("\nDataFrame Info:")
print(df.info())

# Filter out users without bios
df = df[df['bio'].notnull()]

# Calculate the length of each bio in words
df['bio_word_count'] = df['bio'].str.split().str.len()

# Prepare the independent variable (X) and dependent variable (y)
X = df['bio_word_count']
y = df['followers']  # Adjust the column name as per your dataset

# Add a constant to the independent variable (for the intercept)
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X).fit()

# Get the slope (coefficient of the bio_word_count)
slope = model.params['bio_word_count']

# Print the regression slope rounded to three decimal places
print(f"\nRegression slope of followers on bio word count: {slope:.3f}")

DataFrame Overview:
         login              name                     company     location  \
0     cassidoo  Cassidy Williams                      GITHUB  Chicago, IL   
1     felangel     Felix Angelov               SHOREBIRDTECH      Chicago   
2       dabeaz     David Beazley                 DABEAZ, LLC      Chicago   
3  sstephenson    Sam Stephenson                         NaN      Chicago   
4  mattgodbolt      Matt Godbolt  AQUATIC CAPITAL MANAGEMENT  Chicago, IL   

                  email hireable  \
0                   NaN      NaN   
1  felangelov@gmail.com      NaN   
2       dave@dabeaz.com      NaN   
3          sam@sls.name      NaN   
4      matt@godbolt.org      NaN   

                                                 bio  public_repos  followers  \
0            Making memes and dreams... and software           165      13373   
1  software engineer by day, software engineer by...           125       8669   
2  Author of the Python Essential Reference (Addi...     